# Vowpal Wabbit в NLP
Автоматическая обработка текстов - 2017, семинар 4.

В этом семинаре мы познакомимся с библиотекой Vowpal Wabbit и решим с его помощью задачу многоклассовой классификации на больших данных. 
Данные скачайте [здесь](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data) или запустите следующие две ячейки.

! wget https://www.dropbox.com/s/r0q0p0uprhcp8bb/train-sample.zip
! unzip train-sample.zip

! wget https://www.dropbox.com/s/50vw2gsglc91f6o/train.zip
! unzip train.zip

Чтобы на семинаре не тратить время на обработку и обучение моделей на всех данных, предлагается использовать только небольшую подвыборку (`train-sample.csv`). Но сдавать ноутбук все равно необходимо с результатами на **всех** данных.

In [1]:
import numpy as np

In [12]:
import csv

#INPUT_DATA = 'train.csv'
INPUT_DATA = 'train-sample.csv'

reader = csv.DictReader(open(INPUT_DATA))
dict(next(reader))

{'BodyMarkdown': 'I am building a corpus of indexed sentences in different languages. I have a collection of Languages which have both an ObjectId and the ISO code as a key. Is it better to use a reference to the Language collection or store a key like "en" or "fr"?\r\n\r\nI suppose it\'s a compromise between:\r\n\r\n - ease of referencing the Language\r\n - object in that collection\r\n - speed in doing queries where the sentence has a certain language\r\n - the size of the data on disk\r\n\r\nAny best practices that I should know of?',
 'OpenStatus': 'open',
 'OwnerCreationDate': '09/17/2010 10:15:06',
 'OwnerUndeletedAnswerCountAtPostTime': '2',
 'OwnerUserId': '543315',
 'PostClosedDate': '',
 'PostCreationDate': '05/18/2011 14:14:05',
 'PostId': '6046168',
 'ReputationAtPostCreation': '1',
 'Tag1': 'mongodb',
 'Tag2': '',
 'Tag3': '',
 'Tag4': '',
 'Tag5': '',
 'Title': 'For Mongodb is it better to reference an object or use a natural String key?'}

Каждый объект выборки соответствует некоторому посту на Stack Overflow. Требуется построить модель, определяющую статус поста. Подробнее про задачу и формат данных можно прочитать на [странице соревнования](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow).

Перед обучением модели из Vowpal Wabbit данные следует сохранить в специальный формат: <br>
`label |namespace1 feature1:value1 feature2 feature3:value3 |namespace2 ...` <br>
Записи `feature` и `feature:1.0` эквивалентны. Выделение признаков в смысловые подгруппы (namespaces) позволяет создавать взаимодействия между ними. Подробнее про формат входных данных можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Input-format).

Ниже реализована функция, которая извлекает признаки с помощью подаваемого на вход экстрактора, разбивает данные на трейн и тест и записывает их на диск.

In [13]:
STATUSES = ['not a real question', 'not constructive', 'off topic', 'open', 'too localized']
STATUS_DICT = {status: i+1 for i, status in enumerate(STATUSES)}

def data2vw(features_extractor, train_output='train', test_output='test', ytest_output='ytest'):
    reader = csv.DictReader(open(INPUT_DATA))
    writer_train = open(train_output, 'w')
    writer_test = open(test_output, 'w')
    writer_ytest = open(ytest_output, 'w')
    
    for row in reader:
        label = STATUS_DICT[row['OpenStatus']]
        features = features_extractor(row)
        output_line = '%s %s\n' % (label, features)
        if int(row['PostId']) % 2 == 0:
            writer_train.write(output_line)
        else:
            writer_test.write(output_line)
            writer_ytest.write('%s\n' % label)
            
    writer_train.close()
    writer_test.close()
    writer_ytest.close()

Начнем с простейшей модели. В качестве признаков возьмите заголовки и очистите их: приведите символы к нижнему регистру, удалите пунктуацию. Также приветствуется использование стеммеров/лемматизаторов, однако учтите, что они могут сильно замедлить скорость обработки.

In [14]:
import re

def extract_title(row):
    title = row['Title']
    # YOUR CODE HERE
    return ''.join([character for character in title.lower() if (character.isalpha() or character == ' ')])

In [15]:
data2vw(lambda row: '| %s' % extract_title(row))

In [16]:
! head -n 5 train

4 | for mongodb is it better to reference an object or use a natural string key
4 | springdata mongodb querying multiple classes stored in the same collection
4 | stop ajax function in midway when other element is clicked
1 | list of all txt file
5 | i want to design an invitation card for my wedding in silverlight 


Обучим `vw` модель. Параметр `-d` отвечает за путь к обучающей выборке, `-f` – за путь к модели, `--oaa` – за режим мультиклассовой классификации `one-against-all`. Подробное описание всех параметров можно найти [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments) или вызвав `vw --help`.

In [17]:
! vw -d train --loss_function logistic --oaa 5 -f model

final_regressor = model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = train
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4        1       16
0.500000 0.000000            2            2.0        4        4       11
0.500000 0.500000            4            4.0        1        4        6
0.750000 1.000000            8            8.0        3        4        8
0.625000 0.500000           16           16.0        2        4        7
0.500000 0.375000           32           32.0        2        4       10
0.468750 0.437500           64           64.0        1        4       11
0.476562 0.484375          128          128.0        3        4        9
0.511719 0.546875          256          256.0        1        4       10
0.500000 0.488281          512          512.0   

Применим модель к тестовой выборке и сохраним предсказания.

In [18]:
! vw -i model -t test -r pred

only testing
raw predictions = pred
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        4        4       11
0.000000 0.000000            2            2.0        4        4        5
0.250000 0.500000            4            4.0        4        4        8
0.375000 0.500000            8            8.0        3        1        9
0.375000 0.375000           16           16.0        4        4        9
0.406250 0.437500           32           32.0        4        4       13
0.453125 0.500000           64           64.0        3        1       11
0.414062 0.375000          128          128.0        1        4       10
0.406250 0.398438          256          256.0        4        4        9
0.412109 0.417969          512       

In [19]:
! head -n 3 pred

1:-1.33599 2:-4.18829 3:-5.10424 4:1.28751 5:-4.24916
1:-1.33252 2:-2.2637 3:-1.84231 4:0.320449 5:-3.03934
1:0.1727 2:-1.91199 3:-3.26753 4:-1.68744 5:-0.847286


Реализуйте функцию, которая вычисляет `logloss` и `accuracy`, не загружая вектора в память. Используйте `softmax`, чтобы получить вероятности.

In [20]:
def get_scores(ytest_input='ytest', pred_input='pred'):
    n, error, loss = 0, 0, 0
    reader_ytest = open(ytest_input, 'r')
    reader_pred = open(pred_input, 'r')
    
    for label, pred in zip(reader_ytest, reader_pred):
        # YOUR CODE HERE
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def dict_max(dictionary):  
            values = list(dictionary.values())
            keys = list(dictionary.keys())
            return keys[values.index(max(values))]

        label = int(label)
        probabilities = {
            int(item[:item.find(':')]): sigmoid(float(item[item.find(':') + 1:]))
            for item in pred.split(' ')
        }
        loss += np.log(probabilities[label])
        if (dict_max(probabilities) != label):
            error += 1
        n += 1

    reader_ytest.close()
    reader_pred.close()
    return - loss / n, 1 - float(error) / n

In [21]:
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 1.06088
accuracy = 0.59267


На оригинальных данных `logloss` должен быть меньше `0.20`, `accuracy` больше `0.95`. Если это не так, то скорее всего у вас ошибка.

Теперь попробуем улучшить модель, добавив новые признаки, порождаемые словами. В `vowpal wabbit` есть возможность делать это прямо на лету. Воспользуйтесь параметрами `affix`, `ngram`, `skips`.

Далее везде при подборе параметров ориентируйтесь на улучшение `logloss`. Используйте `--quiet` или `-P`, чтобы избавиться от длинных выводов при обучении и применении моделей.

In [32]:
# YOUR CODE HERE
! vw -d train --loss_function logistic --oaa 5 -f model --ngram 1 --ngram 2 --ngram 3 --ngram 4 --quiet
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 1.16406
accuracy = 0.56745


Часто качество `vw` модели получается учушить увеличением числа проходов по обучающей выборке (параметр `--passes`) и увеличением числа бит хэш-функции для уменьшения числа коллизий признаков (параметр `-b`). Подробнее про то, где в `vowpal wabbit` используется хэш-функция, можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Feature-Hashing-and-Extraction). Как меняется качество при изменении этих параметров? Верно ли, что при увеличении значений параметров `--passes` и `-b` качество всегда не убывает и почему?

In [33]:
# YOUR CODE HERE
! vw -d train --loss_function logistic --oaa 5 -f model --ngram 1 --ngram 2 --ngram 3 --ngram 4 --quiet --passes 5 --cache_file cache_file -b 16
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 2.29411
accuracy = 0.50724


Теперь интерес представляет то, какие признаки оказались наиболее важными для модели. Для этого сначала переведем модель в читаемый формат.

In [34]:
! vw -i model -t --invert_hash model.readable train --quiet
! head -n 30 model.readable

Version 8.4.0
Id 
Min label:-50
Max label:50
bits:16
lda:0
4 ngram:1 2 3 4 
0 skip:
options: --oaa 5
Checksum: 3546572062
:0
Constant:10976:-2.67334
Constant[1]:10977:-3.63295
Constant[2]:10978:-3.00891
Constant[3]:10979:2.62512
Constant[4]:10980:-2.82301
a:19856:0.772795
a[1]:19857:0.661652
a[2]:19858:0.65739
a[3]:19859:-0.316702
a[4]:19860:1.12167
a^a:46304:-0.28654
a^a[1]:46305:-0.183126
a^a[2]:46306:-0.335523
a^a[3]:46307:0.0683122
a^a[4]:46308:-0.481479
a^a^directory:14304:-0.134231
a^a^directory[1]:14305:-0.24279
a^a^directory[2]:14306:-0.00054518
a^a^directory[3]:14307:0.174285


Первые несколько строк соответствуют информации о модели. Далее следуют строчки вида `feature[label]:hash:weight`. Выделите для каждого класса 10 признаков с наибольшими по модулю весами. Постарайтесь сделать ваш алгоритм прохода по файлу константным по памяти. Например, можно воспользоваться [кучей](https://docs.python.org/2/library/heapq.html).

In [35]:
# YOUR CODE HERE
import re
from heapq import heappop, heappush

with open('model.readable') as file:
    counter = 0
    label_regular = re.compile(r'\[\d\]')
    feature_regular = re.compile(r'^(.+):\d+:')
    weight_reqular = re.compile(r':\d+:(.+)$')
    heaps = {i: [] for i in STATUS_DICT.values()}
    for line in file:
        if counter > 10:
            label = label_regular.findall(line)
            if len(label) != 0:
                label = int(label[0][1:-1])
            else:
                label = None
            feature = feature_regular.findall(line)[0]
            if feature[-1] == ']':
                feature = feature[:-3]
            weight = float(weight_reqular.findall(line)[0])

            if label is not None:
                if ((weight, feature) not in heaps[label]):
                    heappush(heaps[label], (weight, feature))
                    if len(heaps[label]) > 10:
                        heappop(heaps[label])
                else:
                    print('!!!')
            if counter == 5000:
                break
        counter += 1

In [36]:
heaps[1]

[(0.584254, 'a^break^of^over'),
 (0.636991, 'a^appengine^datastore^element'),
 (0.651453, 'a^basic^bashrc^file'),
 (0.636991, 'a^bar^like'),
 (0.647641, 'a^bit^int^to'),
 (1.5843, 'a^background^location'),
 (0.788221, 'a^ball^so^when'),
 (0.661652, 'a'),
 (0.944992, 'a^bot^script'),
 (0.802438, 'a^badge^precognitive')]

Добавим признаки, извлеченные из текста поста (поле `BodyMarkdown`). В этом поле находится более подробная информация о вопросе, и часто туда помещают код, формулы и т.д. При удалении пунктуации мы потеряем много полезной информации, однако модель "мешка слов" на сырых данных может сильно раздуть признаковое пространство. В таких случаях работают с n-граммами на символах. <br>
Будьте осторожны: символы "`:`" и "`|`" нельзя использовать в названиях признаков, поскольку они являются служебными для `vw`-формата. Замените эти символы на два других редко встречающихся в выборке (или вообще не встречающихся). Также не забудьте про "`\n`". <br>
Поскольку для каждого документа одна n-грамма может встретиться далеко не один раз, то будет экономнее записывать признаки в формате `[n-грамма]:[число вхождений]`.

Также добавьте тэги (поля вид `TagN`). Приветствуется добавление информации о пользователе из других полей. Только не используйте `PostClosedDate` – в нем содержится информация о таргете.

In [76]:
from collections import Counter
import re


def extract_ngram_body(row, ngram=3):
    # YOUR CODE HERE
    body_text = re.sub('[\|\:]', '', row['BodyMarkdown'])
    body_text = re.sub('\n', ' ', body_text)
    def ngram_from_word(counter, word, ngram=2):
        for index in range(1, len(word) - ngram + 1):
            counter[word[index:(index + ngram)]] += 1
    ngrams_dict = Counter()
    for word in body_text.split(' '):
        ngram_from_word(ngrams_dict, word, ngram)
    return ' '.join([key + ':' + str(ngrams_dict[key]) for key in ngrams_dict])


def extract_tags(row):
    # YOUR CODE HERE
    return ' '.join(row['Tag' + str(i)] for i in range(1, 6))

Объединим все вместе. Реализуйте экстрактор признаков, который выделяет каждую подгруппу в отдельный namespace.

In [77]:
extractors_list = [
    ('t', extract_title),
    ('b', extract_ngram_body),
    ('a', extract_tags)
] # (namespace, extractor)


def make_feature_extractor(extractors_list):
    def feature_extractor(row):
        # YOUR CODE HERE
        return ' '.join(['|' + name + ' ' + extractor(row) for name, extractor in extractors_list])
    return feature_extractor

In [ ]:
data2vw(make_feature_extractor(extractors_list))

In [45]:
! head -n 1 train

:1 Ida:1 r"?:1 dat:1 en":1 exe:1 eof:2 age:5 ofi:1 col:3 ofr:1 cet:1 ise:1 oft:1 siz:1 mpr:1 the:6 tao:1 tha:3 |a mongodb    


In [48]:
! vw -d train --loss_function logistic --oaa 5 -f model --quiet

In [49]:
! vw -i model -t test -r pred --quiet
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 1.07433
accuracy = 0.59500


Поэкспериментируйте с другими параметрами модели. Добавьте квадратичные взаимодействия между различными блоками признаков, измените параметры оптимизатора, добавьте регуляризацию и т.д. Совсем необязательно перебирать все параметры по сетке и добиваться оптимального качества. Для нас важнее то, насколько хорошо вы разобрались с возможностями библиотеки и продемонстрировали это.

Выберите не менее трех параметров. Для каждого из них объясните, почему по вашему мнению его изменение может улучшить качество модели, подберите оптимальное значение. Можете перебрать несколько значений "руками", а можете воспользоваться [vw-hypersearch](https://github.com/JohnLangford/vowpal_wabbit/wiki/Using-vw-hypersearch) или [vw-hyperopt](https://github.com/JohnLangford/vowpal_wabbit/blob/master/utl/vw-hyperopt.py) ([статья на хабре](https://habrahabr.ru/company/dca/blog/272697/)). Какие параметры повлияли на улучшение качества сильнее всего?

In [ ]:
# YOUR CODE HERE